In [ ]:
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import torch.utils.data
import cv2
import torchvision.models.segmentation
import torch
import os
import patchify
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image as skimg
import imgaug as ia
import imgaug.augmenters as iaa
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models.detection.mask_rcnn import MaskRCNN_ResNet50_FPN_V2_Weights
import torch.optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.insert(1, '/home/prakharug/AFO')
sys.path.insert(1, '/home/prakharug/AFO/pycoco')
from pycoco.engine import train_one_epoch, evaluate
import miou_eval as mi

In [ ]:
nml = []
for im_name in os.listdir('./dataset11/images'):
    nml.append(im_name)
    #print(im_name)

In [ ]:

start = 1000
for gjgjg in range(10):
    for im_name in nml: 
        rtrnd=random.randint(0,4)
        flipr = random.randint(0,1)
        flipu = random.randint(0,1)
        #perval = random.randint(1,150)/1000
        #alphar = random.randint(0,7000)/100
        seq = iaa.Sequential([
        # iaa.Sometimes(
        #     0.5,
        #     iaa.GaussianBlur(sigma=(0, 0.5))
        # ),
        #iaa.ElasticTransformation(alpha=(alphar,alphar), sigma=5.0),
        #iaa.PerspectiveTransform(scale=(perval, perval),keep_size=True),
        iaa.Rot90((rtrnd), keep_size=False),
        iaa.Fliplr(flipr),
        iaa.Flipud(flipu),
        ])
        tmplist = []
        lent = len(im_name[:-4])
        #print(im_name,im_name[:lent])
        imgtmp = cv2.imread('./dataset2/images/'+im_name,1)
        tmplist.append(imgtmp)
        #tmplist.append(imgtmp[1])
        #tmplist.append(imgtmp[2])
        for gt_name in os.listdir("./dataset2/ground_truths"):
            if gt_name[0:0+lent]==im_name[:lent] and gt_name[0+lent]=="_":
                mask = np.array([cv2.imread("./dataset2/ground_truths/"+gt_name,0)]).transpose(1,2,0)
                #print(mask.shape)
                tmplist.append(mask)
        #tmplist = np.array(tmplist).transpose(1,2,0)
        #print("Tmplist: ",tmplist.shape)
        patch_stack = seq(images = tmplist)
        #patch_stack = np.array(patch_stack).transpose(2,1,0)
        print("Patch stack shape ",len(patch_stack))
        k_w = random.randint(700,patch_stack[1].shape[1])
        k_h = random.randint(700,patch_stack[1].shape[0])
        o_w = random.randint(0,patch_stack[1].shape[1]-k_w)
        o_h = random.randint(0,patch_stack[1].shape[0]-k_h)
        patch_img = np.array(patch_stack[0]).transpose(2,0,1)[:,o_h:o_h+k_h,o_w:o_w+k_w]
        inst = []
        for id in range(1,len(patch_stack)):
            inst.append(patch_stack[id].squeeze())
        inst = np.array(inst)[:,o_h:o_h+k_h,o_w:o_w+k_w].transpose(1,2,0)
        #instances = patch_stack[3:,o_h:o_h+k_h,o_w:o_w+k_w]
        instances = inst
        data = {}
        masks = []
        boxes = []
        area = []
        t=0
        for a in range(instances.shape[2]):
            dispim = instances[:,:,a]
            if np.all(dispim == 0):
                continue
            x,y,w,h = cv2.boundingRect(dispim)
            boxes.append([x, y, x+w, y+h])
            area.append(torch.tensor(h*w))
            masks.append(np.array([dispim]))
            t=1
        if t==0:
            continue
        seq2 = iaa.Sequential()
        cv2.imwrite("./dataset3/images/"+str(start)+'.png',patch_img.transpose(1,2,0))
        mn = 1
        for mks in masks:
            cv2.imwrite("./dataset3/ground_truths/"+str(start)+'_'+str(mn)+'.png',mks.transpose(1,2,0))
            mn+=1

        start+=1
        # masks = torch.as_tensor(masks, dtype=torch.uint8)
        # boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # area = torch.as_tensor(area, dtype=torch.float32)
        # img = torch.as_tensor(patch_img, dtype=torch.float32)
        # img = img/255
        # data["boxes"] =  boxes
        # iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        # data["iscrowd"] = iscrowd
        # data["labels"] =  torch.ones((boxes.shape[0],), dtype=torch.int64)   # there is only one class
        # data["masks"] = masks
        # data["area"] = area
        # data["image_id"] = torch.tensor(idx)

In [ ]:
start = 1000
for gjgjg in range(20):
    for im_name in nml: 
        rtrnd=random.randint(0,4)
        flipr = random.randint(0,1)
        flipu = random.randint(0,1)
        #perval = random.randint(1,150)/1000
        #alphar = random.randint(0,7000)/100
        seq = iaa.Sequential([
        # iaa.Sometimes(
        #     0.5,
        #     iaa.GaussianBlur(sigma=(0, 0.5))
        # ),
        #iaa.ElasticTransformation(alpha=(alphar,alphar), sigma=5.0),
        #iaa.PerspectiveTransform(scale=(perval, perval),keep_size=True),
        iaa.Rot90((rtrnd), keep_size=False),
        iaa.Fliplr(flipr),
        iaa.Flipud(flipu),
        ])
        seq = seq.to_deterministic()
        tmplist = []
        lent = len(im_name[:-4])
        #print(im_name,im_name[:lent])
        imgtmp = cv2.imread('./dataset2/images/'+im_name,1)
        tmplist.append(imgtmp)
        #tmplist.append(imgtmp[1])
        #tmplist.append(imgtmp[2])
        for gt_name in os.listdir("./dataset2/ground_truths"):
            if gt_name[0:0+lent]==im_name[:lent] and gt_name[0+lent]=="_":
                mask = np.array([cv2.imread("./dataset2/ground_truths/"+gt_name,0)]).transpose(1,2,0)
                #print(mask.shape)
                tmplist.append(mask)
        #tmplist = np.array(tmplist).transpose(1,2,0)
        #print("Tmplist: ",tmplist.shape)
        patch_stack = seq(images=tmplist)
        #patch_stack = np.array(patch_stack).transpose(2,1,0)
        print("Patch stack shape ",len(patch_stack))
        k_w = random.randint(512,patch_stack[1].shape[1])
        k_h = random.randint(512,patch_stack[1].shape[0])
        o_w = random.randint(0,patch_stack[1].shape[1]-k_w)
        o_h = random.randint(0,patch_stack[1].shape[0]-k_h)
        patch_img = np.array(patch_stack[0]).transpose(2,0,1)[:,o_h:o_h+k_h,o_w:o_w+k_w]
        inst = []
        for id in range(1,len(patch_stack)):
            inst.append(patch_stack[id].squeeze())
        inst = np.array(inst)[:,o_h:o_h+k_h,o_w:o_w+k_w].transpose(1,2,0)
        #instances = patch_stack[3:,o_h:o_h+k_h,o_w:o_w+k_w]
        instances = inst
        data = {}
        masks = []
        boxes = []
        area = []
        t=0
        for a in range(instances.shape[2]):
            dispim = instances[:,:,a]
            if np.all(dispim == 0):
                continue
            x,y,w,h = cv2.boundingRect(dispim)
            boxes.append([x, y, x+w, y+h])
            area.append(torch.tensor(h*w))
            masks.append(np.array([dispim]))
            t=1
        if t==0:
            continue
        seq2 = iaa.Sequential()
        cv2.imwrite("./dataset4/images/"+str(start)+'.png',patch_img.transpose(1,2,0))
        mn = 1
        for mks in masks:
            cv2.imwrite("./dataset4/ground_truths/"+str(start)+'_'+str(mn)+'.png',mks.transpose(1,2,0))
            mn+=1

        start+=1
        # masks = torch.as_tensor(masks, dtype=torch.uint8)
        # boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # area = torch.as_tensor(area, dtype=torch.float32)
        # img = torch.as_tensor(patch_img, dtype=torch.float32)
        # img = img/255
        # data["boxes"] =  boxes
        # iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        # data["iscrowd"] = iscrowd
        # data["labels"] =  torch.ones((boxes.shape[0],), dtype=torch.int64)   # there is only one class
        # data["masks"] = masks
        # data["area"] = area
        # data["image_id"] = torch.tensor(idx)

In [4]:
start=1000
for kk in range(10):
    for im_name in nml: 
        rtrnd=random.randint(0,4)
        flipr = random.randint(0,1)
        flipu = random.randint(0,1)
        #perval = random.randint(1,150)/1000
        #alphar = random.randint(0,7000)/100
        seq = iaa.Sequential([
        # iaa.Sometimes(
        #     0.5,
        #     iaa.GaussianBlur(sigma=(0, 0.5))
        # ),
        #iaa.ElasticTransformation(alpha=(alphar,alphar), sigma=5.0),
        #iaa.PerspectiveTransform(scale=(perval, perval),keep_size=True),
        iaa.Rot90((rtrnd), keep_size=False),
        iaa.Fliplr(flipr),
        iaa.Flipud(flipu),
        ])
        tmplist = []
        lent = len(im_name[:-4])
        #print(im_name,im_name[:lent])
        imgtmp = cv2.imread('./dataset11/images/'+im_name,1)
        tmplist.append(imgtmp)
        #tmplist.append(imgtmp[1])
        #tmplist.append(imgtmp[2])
        for gt_name in os.listdir("./dataset11/ground_truths"):
            if gt_name[0:0+lent]==im_name[:lent] and gt_name[0+lent]=="_":
                mask = np.array([cv2.imread("./dataset11/ground_truths/"+gt_name,0)]).transpose(1,2,0)
                #print(mask.shape)
                tmplist.append(mask)
        #tmplist = np.array(tmplist).transpose(1,2,0)
        #print("Tmplist: ",tmplist.shape)
        patch_stack = tmplist
        #patch_stack = np.array(patch_stack).transpose(2,1,0)
        print("Patch stack shape ",len(patch_stack))
        k_w = random.randint(700,patch_stack[1].shape[1])
        k_h = random.randint(700,patch_stack[1].shape[0])
        o_w = random.randint(0,patch_stack[1].shape[1]-k_w)
        o_h = random.randint(0,patch_stack[1].shape[0]-k_h)
        patch_stack = seq(images=patch_stack)
        patch_img = np.array(patch_stack[0]).transpose(2,0,1)[:,o_h:o_h+k_h,o_w:o_w+k_w]
        inst = []
        for id in range(1,len(patch_stack)):
            inst.append(patch_stack[id].squeeze())
        inst = np.array(inst)[:,o_h:o_h+k_h,o_w:o_w+k_w].transpose(1,2,0)
        #instances = patch_stack[3:,o_h:o_h+k_h,o_w:o_w+k_w]
        instances = inst
        data = {}
        masks = []
        boxes = []
        area = []
        t=0
        for a in range(instances.shape[2]):
            dispim = instances[:,:,a]
            if np.all(dispim == 0):
                continue
            x,y,w,h = cv2.boundingRect(dispim)
            boxes.append([x, y, x+w, y+h])
            area.append(torch.tensor(h*w))
            masks.append(np.array([dispim]))
            t=1
        if t==0:
            continue
        seq2 = iaa.Sequential()
        cv2.imwrite("./dataset12/images/"+str(start)+'.png',patch_img.transpose(1,2,0))
        mn = 1
        for mks in masks:
            cv2.imwrite("./dataset12/ground_truths/"+str(start)+'_'+str(mn)+'.png',mks.transpose(1,2,0))
            mn+=1

        start+=1
        # masks = torch.as_tensor(masks, dtype=torch.uint8)
        # boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # area = torch.as_tensor(area, dtype=torch.float32)
        # img = torch.as_tensor(patch_img, dtype=torch.float32)
        # img = img/255
        # data["boxes"] =  boxes
        # iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        # data["iscrowd"] = iscrowd
        # data["labels"] =  torch.ones((boxes.shape[0],), dtype=torch.int64)   # there is only one class
        # data["masks"] = masks
        # data["area"] = area
        # data["image_id"] = torch.tensor(idx)

Patch stack shape  23
Patch stack shape  18
Patch stack shape  32
Patch stack shape  16
Patch stack shape  32
Patch stack shape  18
Patch stack shape  25
Patch stack shape  22
Patch stack shape  26
Patch stack shape  20
Patch stack shape  17
Patch stack shape  12
Patch stack shape  17
Patch stack shape  25
Patch stack shape  19
Patch stack shape  18
Patch stack shape  26
Patch stack shape  13
Patch stack shape  22
Patch stack shape  19
Patch stack shape  21
Patch stack shape  22
Patch stack shape  23
Patch stack shape  15
Patch stack shape  15
Patch stack shape  18
Patch stack shape  19
Patch stack shape  22
Patch stack shape  21
Patch stack shape  22
Patch stack shape  35
Patch stack shape  26
Patch stack shape  14
Patch stack shape  16
Patch stack shape  20
Patch stack shape  23
Patch stack shape  20
Patch stack shape  18
Patch stack shape  26
Patch stack shape  22
Patch stack shape  18
Patch stack shape  22
Patch stack shape  32
Patch stack shape  18
Patch stack shape  21
Patch stac